<a href="https://colab.research.google.com/github/Crossme0809/langchain-tutorials/blob/main/Using_Falcon7B_And_LangChain_Build_Conversation_Memory_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Aug  4 10:03:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq git+https://github.com/huggingface/transformers --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq langchain==0.0.233 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import re
import warnings
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

warnings.filterwarnings("ignore", category=UserWarning)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

## 加载模型

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, device_map="auto", quantization_config=quantization_config
)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Model device: {model.device}")

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Model device: cuda:0


In [ ]:
!nvidia-smi

Fri Aug  4 10:13:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    29W /  70W |   4825MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 256,
  "pad_token_id": 11,
  "repetition_penalty": 1.7,
  "temperature": 0,
  "transformers_version": "4.32.0.dev0",
  "use_cache": false
}

In [ ]:
model.config

FalconConfig {
  "_name_or_path": "vilsonrodrigues/falcon-7b-instruct-sharded",
  "alibi": false,
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "FalconForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "vilsonrodrigues/falcon-7b-instruct-sharded--configuration_falcon.FalconConfig",
    "AutoModel": "vilsonrodrigues/falcon-7b-instruct-sharded--modeling_falcon.FalconModel",
    "AutoModelForCausalLM": "vilsonrodrigues/falcon-7b-instruct-sharded--modeling_falcon.FalconForCausalLM",
    "AutoModelForQuestionAnswering": "vilsonrodrigues/falcon-7b-instruct-sharded--modeling_falcon.FalconForQuestionAnswering",
    "AutoModelForSequenceClassification": "vilsonrodrigues/falcon-7b-instruct-sharded--modeling_falcon.FalconForSequenceClassification",
    "AutoModelForTokenClassification": "vilsonrodrigues/falcon-7b-instruct-sharded--modeling_falcon.FalconForTokenClassification"
  },
  "bias": false,
  "bos_token_id": 11,
  "eos_token_id":

In [106]:
prompt = """
以下是一个人类和AI之间友好的对话。AI非常健谈，并提供了许多具体的细节。

当前对话：

Human：Dwight K Schrute是谁？
AI：
""".strip()
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

In [107]:
%%time

with torch.inference_mode():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
    )

CPU times: user 5.63 s, sys: 21.9 ms, total: 5.65 s
Wall time: 11.7 s


In [108]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

以下是一个人类和AI之间友好的对话。AI非常健谈，并提供了许多具体的细节。

当前对话：

Human：Dwight K Schrute是谁？
AI：Dwight K Schrute is a fictional character from the TV show "The Office". He is played by Rainn Wilson.
User 


## 阻止LLM胡言乱语
LLM生成的内容经常会偏离主题，生成与问题无关或毫无意义的回复。虽然这是一个持续的研究挑战，但作为LLM在实际应用中的用户，我们可以通过一些方法来解决这个问题。我们将使用一种名为StoppingCriteria的技术来控制输出，防止模型胡言乱语或产生虚构的问题和对话：

In [110]:
class StopGenerationCriteria(StoppingCriteria):
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False

In [111]:
stop_tokens = [["Human", ":"], ["AI", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)

In [112]:
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

In [113]:
res = llm(prompt)
print(res)

Dwight K Schrute is a fictional character from the TV show "The Office". He is played by Rainn Wilson.
User 


## Conversation Chain

In [114]:
chain = ConversationChain(llm=llm)
print(chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


### Custom Prompt

In [115]:
template = """
以下是一个人类和AI之间的对话。这个AI的行为方式完全像《办公室》电视剧中的Dwight K Schrute一样。Dwight是一位经验丰富且非常成功的在线营销人员和销售员。他机智、有说服力、直接而实际。Dwight会帮助完成分配给他的每个营销任务。如果Dwight不知道某个问题的答案，他会真诚地说自己不知道。

当前对话：
{history}
Human：{input}
AI:""".strip()

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

print(template)

以下是一个人类和AI之间的对话。这个AI的行为方式完全像《办公室》电视剧中的Dwight K Schrute一样。Dwight是一位经验丰富且非常成功的在线营销人员和销售员。他机智、有说服力、直接而实际。Dwight会帮助完成分配给他的每个营销任务。如果Dwight不知道某个问题的答案，他会真诚地说自己不知道。

当前对话：
{history}
Human：{input}
AI:


In [116]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [117]:
text = "为那些配备大型V8发动机的家庭汽车制造商想一个名字。这个名字必须是一个单词，并且容易发音。"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
以下是一个人类和AI之间的对话。这个AI的行为方式完全像《办公室》电视剧中的Dwight K Schrute一样。Dwight是一位经验丰富且非常成功的在线营销人员和销售员。他机智、有说服力、直接而实际。Dwight会帮助完成分配给他的每个营销任务。如果Dwight不知道某个问题的答案，他会真诚地说自己不知道。

当前对话：

Human：为那些配备大型V8发动机的家庭汽车制造商想一个名字。这个名字必须是一个单词，并且容易发音。
AI:

> Finished chain.
 "V8"
User 


### Cleaning Output

In [118]:
class CleanupOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        user_pattern = r"\nUser"
        text = re.sub(user_pattern, "", text)
        human_pattern = r"\nHuman:"
        text = re.sub(human_pattern, "", text)
        ai_pattern = r"\nAI:"
        return re.sub(ai_pattern, "", text).strip()

    @property
    def _type(self) -> str:
        return "output_parser"

In [119]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    output_parser=CleanupOutputParser(),
    verbose=True,
)

## 与AI聊天

In [120]:
text = "为那些配备大型V8发动机的家庭汽车制造商想一个名字。这个名字必须是一个单词，并且容易发音。"
res = chain(text)



> Entering new ConversationChain chain...
Prompt after formatting:
以下是一个人类和AI之间的对话。这个AI的行为方式完全像《办公室》电视剧中的Dwight K Schrute一样。Dwight是一位经验丰富且非常成功的在线营销人员和销售员。他机智、有说服力、直接而实际。Dwight会帮助完成分配给他的每个营销任务。如果Dwight不知道某个问题的答案，他会真诚地说自己不知道。

当前对话：

Human：为那些配备大型V8发动机的家庭汽车制造商想一个名字。这个名字必须是一个单词，并且容易发音。
AI:

> Finished chain.


In [121]:
res.keys()

dict_keys(['input', 'history', 'response'])

In [122]:
print(res["response"])

"V8"


In [123]:
text = "为公司想一个口号"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
以下是一个人类和AI之间的对话。这个AI的行为方式完全像《办公室》电视剧中的Dwight K Schrute一样。Dwight是一位经验丰富且非常成功的在线营销人员和销售员。他机智、有说服力、直接而实际。Dwight会帮助完成分配给他的每个营销任务。如果Dwight不知道某个问题的答案，他会真诚地说自己不知道。

当前对话：
Human: 为那些配备大型V8发动机的家庭汽车制造商想一个名字。这个名字必须是一个单词，并且容易发音。
AI: "V8"
Human：为公司想一个口号
AI:

> Finished chain.
"V8 Nation"


In [124]:
text = "为公司选择一个域名"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
以下是一个人类和AI之间的对话。这个AI的行为方式完全像《办公室》电视剧中的Dwight K Schrute一样。Dwight是一位经验丰富且非常成功的在线营销人员和销售员。他机智、有说服力、直接而实际。Dwight会帮助完成分配给他的每个营销任务。如果Dwight不知道某个问题的答案，他会真诚地说自己不知道。

当前对话：
Human: 为那些配备大型V8发动机的家庭汽车制造商想一个名字。这个名字必须是一个单词，并且容易发音。
AI: "V8"
Human: 为公司想一个口号
AI: "V8 Nation"
Human：为公司选择一个域名
AI:

> Finished chain.
"v8nation.com"


In [125]:
text = "写一条推文介绍公司，并介绍公司制造的第一辆汽车"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
以下是一个人类和AI之间的对话。这个AI的行为方式完全像《办公室》电视剧中的Dwight K Schrute一样。Dwight是一位经验丰富且非常成功的在线营销人员和销售员。他机智、有说服力、直接而实际。Dwight会帮助完成分配给他的每个营销任务。如果Dwight不知道某个问题的答案，他会真诚地说自己不知道。

当前对话：
Human: 为那些配备大型V8发动机的家庭汽车制造商想一个名字。这个名字必须是一个单词，并且容易发音。
AI: "V8"
Human: 为公司想一个口号
AI: "V8 Nation"
Human: 为公司选择一个域名
AI: "v8nation.com"
Human：写一条推文介绍公司，并介绍公司制造的第一辆汽车
AI:

> Finished chain.
"V8 Nation is the home of the first V8-powered car. We're proud to be a part of this exciting industry and we look forward to providing you with the best service possible."


In [126]:
text = "写一封简短的营销邮件，销售公司的第一辆汽车-一辆配备700马力超级V8发动机和手动变速箱的家庭轿车。"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
以下是一个人类和AI之间的对话。这个AI的行为方式完全像《办公室》电视剧中的Dwight K Schrute一样。Dwight是一位经验丰富且非常成功的在线营销人员和销售员。他机智、有说服力、直接而实际。Dwight会帮助完成分配给他的每个营销任务。如果Dwight不知道某个问题的答案，他会真诚地说自己不知道。

当前对话：
Human: 为那些配备大型V8发动机的家庭汽车制造商想一个名字。这个名字必须是一个单词，并且容易发音。
AI: "V8"
Human: 为公司想一个口号
AI: "V8 Nation"
Human: 为公司选择一个域名
AI: "v8nation.com"
Human: 写一条推文介绍公司，并介绍公司制造的第一辆汽车
AI: "V8 Nation is the home of the first V8-powered car. We're proud to be a part of this exciting industry and we look forward to providing you with the best service possible."
Human：写一封简短的营销邮件，销售公司的第一辆汽车-一辆配备700马力超级V8发动机和手动变速箱的家庭轿车。
AI:

> Finished chain.
"V8 Nation is proud to offer the best in luxury and performance. Our newest addition, the V8 Nation Supercar, is the perfect combination of power and elegance. With its hand-built V8 engine and 6-speed manual transmission, it's the ideal choice for any family that wants to experience the thrill of driving something special. Visit our website to le